### Step 1: Import necessary libraries

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

### Step 2: Prepare data for the model

In [4]:
customers = pd.read_csv("C:/Users/satya/Downloads/Customers (1).csv")
products = pd.read_csv("C:/Users/satya/Downloads/Products (1).csv")
transactions = pd.read_csv("C:/Users/satya/Downloads/Transactions.csv")

customer_transactions = transactions.groupby('CustomerID').agg({
    'Quantity': 'sum', 
    'TotalValue': 'sum'
}).reset_index()

merged_customers = pd.merge(customers, customer_transactions, on='CustomerID', how='left').fillna(0)

### Step 3: Normalize data

In [6]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(merged_customers[['Quantity', 'TotalValue']])

### Step 4: Compute similarity matrix

In [8]:
similarity_matrix = cosine_similarity(scaled_features)
similarity_df = pd.DataFrame(similarity_matrix, index=merged_customers['CustomerID'], columns=merged_customers['CustomerID'])

### Step 5: Find top 3 similar customers

In [10]:
lookalike_dict = {}
for customer in merged_customers['CustomerID'][:20]:  # First 20 customers
    similar_customers = similarity_df[customer].sort_values(ascending=False).iloc[1:4]
    lookalike_dict[customer] = list(zip(similar_customers.index, similar_customers.values))

### Save lookalikes to a CSV

In [12]:
lookalike_df = pd.DataFrame([
    {'CustomerID': cust, 'Lookalikes': lookalike_dict[cust]}
    for cust in lookalike_dict
])
lookalike_df.to_csv('YourName_Lookalike.csv', index=False)

print("Lookalike recommendations saved to 'YourName_Lookalike.csv'")

Lookalike recommendations saved to 'YourName_Lookalike.csv'
